In [1]:
%%bash
pip install timm onnx onnxruntime -q

In [2]:
import os
import sys
import cv2
import json
import onnx
import timm
import torch
import random as r
import numpy as np
import pandas as pd
import onnxruntime as ort
import matplotlib.pyplot as plt

from PIL import Image
from typing import Union
from torchvision import models
from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")

if not os.path.exists("onnx"): os.makedirs("onnx")
    
ort.set_default_logger_severity(3)

In [3]:
labels: dict = json.load(open("/kaggle/input/mfi-dataframe/labels.json", "r"))
    
    
def breaker() -> None:
    print("\n" + 50*"*" + "\n")


def get_image(path: str, size: int=224) -> np.ndarray:
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    return cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    

def show_image(
    image: np.ndarray, 
    cmap: str="gnuplot2", 
    title: Union[str, None]=None
) -> None:
    plt.figure()
    plt.imshow(image, cmap=cmap)
    plt.axis("off")
    if title: plt.title(title)
    plt.show()

In [4]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.model = timm.create_model(model_name="efficientnet_b4", pretrained=False)
        self.model.classifier = torch.nn.Linear(in_features=self.model.classifier.in_features, out_features=5)

    def forward(self, x):
        return self.model(x)

    
class CFG(object):  
    def __init__(
        self, 
        in_channels: int=3, 
        size: int=256, 
        opset_version: int=9, 
        path: str=None
    ):
        self.in_channels = in_channels
        self.size = size
        self.dummy = torch.randn(1, self.in_channels, self.size, self.size)
        self.opset_version = opset_version
        self.path = path

In [5]:
# Fold 3 (BEST), Fold 1(SECOND BEST), Fold 1(THIRD BEST)

for fold in range(1, 6):
    for v in ["a", "l"]:
        cfg = CFG(
            in_channels=3, 
            size=224, 
            opset_version=15, 
            path=f"/kaggle/input/mfi-en4-a224-e10-f{fold}/saves/b{v}e_state_fold_{fold}.pt"
        )

        model = Model()
        model.load_state_dict(torch.load(cfg.path, map_location=torch.device("cpu"))["model_state_dict"])
        model.eval()

        clear_output()

        torch.onnx.export(
            model=model, 
            args=cfg.dummy, 
            f=f"onnx/b{v}e_model_f{fold}.onnx", 
            input_names=["input"], 
            output_names=["output"], 
            opset_version=cfg.opset_version,
            export_params=True,
            training=torch.onnx.TrainingMode.EVAL,
            operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
            dynamic_axes={
              "input"  : {0 : "batch_size"},
              "output" : {0 : "batch_size"},
            }
        )

============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [6]:
class OnnxModel(object):
    def __init__(self, path: str) -> None:
        self.size: int = 224
        
        self.mean: list = [0.54442, 0.53724, 0.48565]
        self.std: list  = [0.06842, 0.06327, 0.05378]
        
        self.path: str = path
    
        model = onnx.load(self.path)
        onnx.checker.check_model(model)
        self.ort_session = ort.InferenceSession(self.path)
        
    def infer(self, image: np.ndarray, labels: dict) -> np.ndarray:
        image = image / 255
        image = cv2.resize(src=image, dsize=(self.size, self.size), interpolation=cv2.INTER_AREA).transpose(2, 0, 1)
        for i in range(image.shape[0]): image[i, :, :] = (image[i, :, :] - self.mean[i]) / self.std[i]
        image = np.expand_dims(image, axis=0)
        inputs = {self.ort_session.get_inputs()[0].name: image.astype("float32")}
        result = self.ort_session.run(None, inputs)
        odds = np.exp(np.max(result))
        return labels[str(np.argmax(result))].title(), odds / (1 + odds)

In [7]:
df = pd.read_csv("/kaggle/input/mfi-dataframe/test.csv")

breaker()
for model_name in sorted(os.listdir("onnx")):
    
    onnx_model = OnnxModel(f"onnx/{model_name}")
    
    print(f"{model_name.upper()}\n")
    
    count: int = 0
    
    for i in range(9):
        index = r.randint(0, len(df)-1)
        filepath = df.iloc[index, 0]
        y_true   = df.iloc[index, 1]

        image = get_image(filepath)

        y_pred, _ = onnx_model.infer(image, labels)
        
        if labels[str(y_true)].title() == y_pred:
            count += 1
        
        print(f"{labels[str(y_true)].title()}, {y_pred}")
        # show_image(image, title=y_pred)
    
    print(f"\n\nAccuracy : {count / 10:.2f}")

    breaker()


**************************************************

BAE_MODEL_F1.ONNX

H2, H2
H2, H2
H2, H2
H1, H1
H1, H1
H6, H6
H2, H2
H1, H1
H1, H3


Accuracy : 0.80

**************************************************

BAE_MODEL_F2.ONNX

H1, H1
H1, H1
H1, H1
H1, H1
H6, H6
H1, H1
H1, H1
H2, H2
H1, H1


Accuracy : 0.90

**************************************************

BAE_MODEL_F3.ONNX

H1, H1
H1, H2
H2, H1
H5, H5
H5, H5
H1, H1
H1, H1
H1, H2
H1, H1


Accuracy : 0.60

**************************************************

BAE_MODEL_F4.ONNX

H1, H1
H6, H5
H1, H1
H1, H1
H3, H3
H6, H6
H3, H3
H1, H1
H1, H1


Accuracy : 0.80

**************************************************

BAE_MODEL_F5.ONNX

H6, H6
H6, H6
H2, H2
H2, H2
H1, H1
H1, H2
H3, H3
H2, H2
H1, H1


Accuracy : 0.80

**************************************************

BLE_MODEL_F1.ONNX

H1, H1
H1, H1
H3, H3
H3, H3
H3, H3
H1, H1
H2, H2
H1, H3
H2, H2


Accuracy : 0.80

**************************************************

BLE_MODEL_F2.ONNX

H5, H5
H1,